# Prepare Active DNS Tunneling Attack Dataset

## Overview:

This notebook will focus on creating an active DNS Tunneling attack dataset based on a small sample of data collected by performing real DNS Tunneling attacks in a controlled environment.<br>
The dataset that this notebook creates closely represents real-world data and was used to train our SVM model.<br>  
It is worth noteing that the sample dataset we collected does not contain any missing values or any outliers due to the fact we tested each part of the collection process and verified that it is correct.<br>
In this notebook we have generated an attack dataset with 40,000 flows of the DNS Tunneling attacks based on the samples we collected when running a DNS Tunneling attacks in various configurations using the well known DNScat2 tool when the victim host has an active DNS tunnel, meaning its open and commands are running in it.<br> 

## Imports & Global Variables:

In [12]:
import pandas as pd
import numpy as np
import random

NUM_OF_ROWS = 40000
ATTACK_NAME = 'DNS'

In [13]:
# the following command will make it so that when we print the dataframe we will see all the columns
pd.set_option('display.max_columns', None)

---

## Load the sample dataset:

In [14]:
# import the attack sample dataset
dns_samples = pd.read_csv('dns_samples_active.csv')
print(f'Dataset Shape: {dns_samples.shape}')
dns_samples

Dataset Shape: (20, 27)


,A Record Count,AAAA Record Count,CName Record Count,TXT Record Count,MX Record Count,DF Flag Count,Average Response Data Length,Min Response Data Length,Max Response Data Length,Average Domain Name Length,Min Domain Name Length,Max Domain Name Length,Average Sub Domain Name Length,Min Sub Domain Name Length,Max Sub Domain Name Length,Average Packet Length,Min Packet Length,Max Packet Length,Number of Domian Names,Number of Sub Domian Names,Total Length of Fwd Packet,Total Length of Bwd Packet,Total Number of Packets,Flow Duration,IAT Max,IAT Mean,IAT Std
0,0,0,150,84,116,174,39.758621,34,70,181.363636,42,241,32.309848,14.333333,39.833333,268.742857,101,386,88,276,44448,34912,350,17.115152,1.102749,0.049041,0.153262
1,0,0,94,68,60,110,39.550000,34,70,149.000000,42,241,28.125000,14.333333,39.833333,233.810811,101,357,56,148,23990,18592,222,14.696482,0.770567,0.066500,0.158759
2,0,0,94,96,136,162,38.127660,34,52,185.609756,42,241,32.740854,14.333333,39.833333,272.122699,101,357,82,262,41792,33228,326,14.316078,0.990149,0.044049,0.145406
3,0,0,100,128,88,158,38.175439,34,70,170.493671,42,241,30.845992,14.333333,39.833333,256.006329,101,357,79,234,38002,29624,316,16.216136,0.999400,0.051480,0.151140
4,0,0,136,122,88,172,38.406250,34,44,207.091954,42,241,35.493295,14.333333,39.833333,292.161850,101,357,87,306,47564,38992,346,11.440871,0.990594,0.033162,0.099968
5,0,0,56,90,68,106,37.388889,34,44,155.870370,42,241,28.910494,14.333333,39.833333,240.252336,101,357,54,149,23756,18670,214,13.661421,0.801823,0.064138,0.165087
6,0,0,74,60,80,106,38.909091,34,52,108.611111,42,241,22.862654,14.333333,39.833333,194.897196,101,357,54,111,19154,13566,214,19.716813,0.987562,0.092567,0.255850
7,0,0,66,52,72,94,39.655172,34,60,132.375000,42,241,25.881944,14.333333,39.833333,219.200000,101,357,48,116,19328,14340,190,13.953869,0.996097,0.073830,0.189524
8,0,0,72,52,80,102,38.645161,34,42,84.529412,42,241,19.767974,14.333333,39.833333,170.578431,101,357,51,86,15874,10356,204,21.172616,0.993989,0.104299,0.275736
9,0,0,60,68,96,112,38.875000,34,60,119.464286,42,241,24.232143,14.333333,39.833333,205.500000,101,357,56,124,21340,15284,224,18.313704,0.979127,0.082124,0.187037


### Find the columns that we need to synthesis data for:

In [15]:
columns_to_gather = dns_samples.replace(0, np.nan) #replace all 0 values with null
columns_to_gather = columns_to_gather.dropna(how = 'all', axis = 1).columns.tolist() #remove all columns where there are null values
columns_to_gather #left with all columns that the values are not 0 (be know for a fact that the data is consistant and there are not missing values in the rows)

['CName Record Count',
 'TXT Record Count',
 'MX Record Count',
 'DF Flag Count',
 'Average Response Data Length',
 'Min Response Data Length',
 'Max Response Data Length',
 'Average Domain Name Length',
 'Min Domain Name Length',
 'Max Domain Name Length',
 'Average Sub Domain Name Length',
 'Min Sub Domain Name Length',
 'Max Sub Domain Name Length',
 'Average Packet Length',
 'Min Packet Length',
 'Max Packet Length',
 'Number of Domian Names',
 'Number of Sub Domian Names',
 'Total Length of Fwd Packet',
 'Total Length of Bwd Packet',
 'Total Number of Packets',
 'Flow Duration',
 'IAT Max',
 'IAT Mean',
 'IAT Std']

### Find an approximate minimum and maximum values of each column:

In [16]:
# find the minimum and maximum values for each column, scale the range (reduce min by 10% and increase max by 15%), and store the results in a dictionary.
min_max_dict = {col: (float(dns_samples[col].min() * 0.9), float(dns_samples[col].max() * 1.15)) for col in columns_to_gather}

# print the min max dictionary
for col, (min_val, max_val) in min_max_dict.items():
    print(f'{col:<30} | Min: {min_val:.2f} | Max: {max_val:.2f}')

CName Record Count             | Min: 37.80 | Max: 172.50
TXT Record Count               | Min: 46.80 | Max: 170.20
MX Record Count                | Min: 48.60 | Max: 156.40
DF Flag Count                  | Min: 84.60 | Max: 200.10
Average Response Data Length   | Min: 33.58 | Max: 45.72
Min Response Data Length       | Min: 30.60 | Max: 39.10
Max Response Data Length       | Min: 37.80 | Max: 80.50
Average Domain Name Length     | Min: 76.08 | Max: 238.16
Min Domain Name Length         | Min: 37.80 | Max: 48.30
Max Domain Name Length         | Min: 216.90 | Max: 277.15
Average Sub Domain Name Length | Min: 17.79 | Max: 40.82
Min Sub Domain Name Length     | Min: 12.90 | Max: 16.48
Max Sub Domain Name Length     | Min: 35.85 | Max: 45.81
Average Packet Length          | Min: 153.52 | Max: 335.99
Min Packet Length              | Min: 90.90 | Max: 116.15
Max Packet Length              | Min: 321.30 | Max: 443.90
Number of Domian Names         | Min: 43.20 | Max: 101.20
Number of Sub Domi

### Create the base attack dataset (full of zeros):

In [18]:
# creating an empty dataframe before adding values to it
dns_dataset = pd.DataFrame(np.zeros((NUM_OF_ROWS, len(dns_samples.columns))), columns = dns_samples.columns)
dns_dataset.head(3)

,A Record Count,AAAA Record Count,CName Record Count,TXT Record Count,MX Record Count,DF Flag Count,Average Response Data Length,Min Response Data Length,Max Response Data Length,Average Domain Name Length,Min Domain Name Length,Max Domain Name Length,Average Sub Domain Name Length,Min Sub Domain Name Length,Max Sub Domain Name Length,Average Packet Length,Min Packet Length,Max Packet Length,Number of Domian Names,Number of Sub Domian Names,Total Length of Fwd Packet,Total Length of Bwd Packet,Total Number of Packets,Flow Duration,IAT Max,IAT Mean,IAT Std
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Find the columns with constant zero values based on samples:

In [19]:
# adding zeros to all columns that should not have any values
zero_columns = [col for col in dns_samples.columns if col not in columns_to_gather]
for col in zero_columns:
    dns_dataset[col] = int(0)
zero_columns

['A Record Count', 'AAAA Record Count']

---

## Filling in values based on collected samples:

### Firstly we insert data into columns that are not related to each other:

In [20]:
constant_columns = ['TXT Record Count', 'MX Record Count', 'CName Record Count', 'DF Flag Count', 
                   'Average Response Data Length', 'Min Response Data Length', 'Max Response Data Length', 
                   'Min Domain Name Length', 'Max Domain Name Length', 
                   'Min Sub Domain Name Length', 'Max Sub Domain Name Length',
                   'Min Packet Length', 'Max Packet Length', 'Average Sub Domain Name Length', 'Flow Duration']

# adding the attack feature values to the dataset at random based on the smaple data using the minimum maximum dict
for col in constant_columns:
    dns_dataset[col] = np.random.randint(min_max_dict[col][0], min_max_dict[col][1], NUM_OF_ROWS)

In [21]:
dns_dataset

,A Record Count,AAAA Record Count,CName Record Count,TXT Record Count,MX Record Count,DF Flag Count,Average Response Data Length,Min Response Data Length,Max Response Data Length,Average Domain Name Length,Min Domain Name Length,Max Domain Name Length,Average Sub Domain Name Length,Min Sub Domain Name Length,Max Sub Domain Name Length,Average Packet Length,Min Packet Length,Max Packet Length,Number of Domian Names,Number of Sub Domian Names,Total Length of Fwd Packet,Total Length of Bwd Packet,Total Number of Packets,Flow Duration,IAT Max,IAT Mean,IAT Std
0,0,0,105,121,116,167,40,33,57,0.0,47,237,20,12,35,0.0,115,398,0.0,0.0,0.0,0.0,0.0,16,0.0,0.0,0.0
1,0,0,53,59,150,142,38,37,55,0.0,41,245,28,12,44,0.0,101,380,0.0,0.0,0.0,0.0,0.0,13,0.0,0.0,0.0
2,0,0,115,146,67,175,37,30,75,0.0,43,257,19,15,41,0.0,105,348,0.0,0.0,0.0,0.0,0.0,16,0.0,0.0,0.0
3,0,0,84,79,93,152,40,36,51,0.0,45,271,21,12,43,0.0,91,336,0.0,0.0,0.0,0.0,0.0,11,0.0,0.0,0.0
4,0,0,100,167,55,134,37,37,57,0.0,37,227,36,15,43,0.0,99,356,0.0,0.0,0.0,0.0,0.0,19,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,0,0,157,69,108,113,43,36,78,0.0,42,260,21,12,40,0.0,111,362,0.0,0.0,0.0,0.0,0.0,16,0.0,0.0,0.0
39996,0,0,141,168,111,144,37,34,42,0.0,37,255,24,12,41,0.0,106,391,0.0,0.0,0.0,0.0,0.0,14,0.0,0.0,0.0
39997,0,0,135,53,124,121,39,30,63,0.0,47,270,26,12,41,0.0,114,331,0.0,0.0,0.0,0.0,0.0,18,0.0,0.0,0.0
39998,0,0,102,128,144,171,44,36,70,0.0,42,274,29,13,42,0.0,91,360,0.0,0.0,0.0,0.0,0.0,12,0.0,0.0,0.0


## Then we fill values into columns that have a certain correlation between them:

A correlation between two or more columns is common in our dataset since most features are inherently related. All of them are derived from network packet traffic.<br>
For example, as the **average sub domain name length** increases, the **number of sub domian names** is likely to increases as well. This happens because of the nature of domain names, the longer the sub domain length, the more sub domains can fit inside it.<br>

### Correlation between 'Average Sub Domain Name Length' and all of the following: 'Average Domain Name Length', 'Number of Sub Domian Names', 'Average Packet Length', 'Number of Domian Names', 'Total Length of Fwd Packet', 'Total Length of Bwd Packet', 'Total Number of Packets':

In [22]:
base_column = 'Average Sub Domain Name Length'
correlation_columns = ['Average Domain Name Length', 'Number of Sub Domian Names', 'Average Packet Length', 
                        'Number of Domian Names', 'Total Length of Fwd Packet', 'Total Length of Bwd Packet', 'Total Number of Packets']

In [23]:
# finding the correlation between the 'Average Sub Domain Name Length' column to the rest of the columns in correlation_columns
independent_col = dns_samples[base_column].values.reshape(-1, 1) #column 'Average Sub Domain Name Length'
dependent_cols = dns_samples[correlation_columns].values

# using least squares regression to find scaling factors that best approximate the relationship between 'Average Sub Domain Name Length' and the rest
scaling_factors = np.linalg.lstsq(independent_col, dependent_cols, rcond = None)[0]
scaling_factors = [(name, float(factor)) for name, factor in zip(correlation_columns, scaling_factors.flatten())]
for val in scaling_factors:
    print(val)

# Precompute values using random deltas for all rows and all dependent columns
for col, factor in scaling_factors:
    deltas = np.random.uniform(factor * 0.075, factor * 0.2, size = NUM_OF_ROWS)
    signs = np.random.choice([-1, 1], size = NUM_OF_ROWS)
    updated_factors = factor + signs * deltas

    # insert the data into the attack dataset for each column
    dns_dataset[col] = dns_dataset[base_column] * updated_factors

('Average Domain Name Length', 5.397201131256436)
('Number of Sub Domian Names', 6.80256261306239)
('Average Packet Length', 8.346997407096115)
('Number of Domian Names', 2.3720454375026234)
('Total Length of Fwd Packet', 1103.566273611197)
('Total Length of Bwd Packet', 856.9022907907998)
('Total Number of Packets', 9.450172899622817)


In [24]:
dns_dataset

,A Record Count,AAAA Record Count,CName Record Count,TXT Record Count,MX Record Count,DF Flag Count,Average Response Data Length,Min Response Data Length,Max Response Data Length,Average Domain Name Length,Min Domain Name Length,Max Domain Name Length,Average Sub Domain Name Length,Min Sub Domain Name Length,Max Sub Domain Name Length,Average Packet Length,Min Packet Length,Max Packet Length,Number of Domian Names,Number of Sub Domian Names,Total Length of Fwd Packet,Total Length of Bwd Packet,Total Number of Packets,Flow Duration,IAT Max,IAT Mean,IAT Std
0,0,0,105,121,116,167,40,33,57,124.640990,47,237,20,12,35,195.141639,115,398,52.087743,161.903345,24968.744358,14004.472088,152.745326,16,0.0,0.0,0.0
1,0,0,53,59,150,142,38,37,55,169.301178,41,245,28,12,44,270.967873,101,380,59.613649,175.924356,28044.682234,28250.244604,294.898059,13,0.0,0.0,0.0
2,0,0,115,146,67,175,37,30,75,121.304754,43,257,19,15,41,137.533201,105,348,52.578456,107.582658,18235.397200,13175.234831,147.316322,16,0.0,0.0,0.0
3,0,0,84,79,93,152,40,36,51,95.355032,45,271,21,12,43,190.074515,91,336,54.129374,154.398343,26968.657308,20971.090058,229.765860,11,0.0,0.0,0.0
4,0,0,100,167,55,134,37,37,57,164.069280,37,227,36,15,43,256.237147,99,356,71.148402,203.192695,36389.583265,25285.933347,384.257905,19,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,0,0,157,69,108,113,43,36,78,95.152610,42,260,21,12,40,195.085999,111,362,41.722544,127.218142,21070.434026,15073.859726,175.568829,16,0.0,0.0,0.0
39996,0,0,141,168,111,144,37,34,42,140.575055,37,255,24,12,41,169.650092,106,391,49.209418,192.871261,24335.411247,24512.009466,259.021497,14,0.0,0.0,0.0
39997,0,0,135,53,124,121,39,30,63,115.138049,47,270,26,12,41,193.516657,114,331,53.211755,158.212101,32922.548683,24974.680187,277.167783,18,0.0,0.0,0.0
39998,0,0,102,128,144,171,44,36,70,140.695768,42,274,29,13,42,202.621234,91,360,79.674437,217.572656,27312.971880,20605.718660,302.169624,12,0.0,0.0,0.0


### Correlation between 'IAT Mean' and all of the following: 'IAT Max', 'IAT Std', 'Flow Duration':

In [ ]:
# finding the correlation between the 'IAT Mean' column to the rest of the columns in order to create new data
correlation = ['IAT Mean', 'IAT Max', 'IAT Std', 'Flow Duration']
independent_col = dns_samples[correlation[0]].values.reshape(-1, 1) #column 'IAT Mean'
dependent_cols = dns_samples[correlation[1:]].values

# using least squares regression to find scaling factors that best approximate the relationship between 'IAT Mean' and the rest of the columns in correlation
scaling_factors = np.linalg.lstsq(independent_col, dependent_cols, rcond = None)[0]

scaling_factors = [(name, float(factor)) for name, factor in zip(correlation[1:], scaling_factors.flatten())]
for val in scaling_factors:
    print(val)

('IAT Max', 13.127911876318981)
('IAT Std', 2.67841979702811)
('Flow Duration', 232.71429250504946)


In [26]:
dns_dataset['IAT Mean'] = np.random.uniform(min_max_dict['IAT Mean'][0], min_max_dict['IAT Mean'][1], NUM_OF_ROWS)

# iterating over all rows we need to add values
for col, factor in scaling_factors: #iterating over all rows we need to add values to except 'IAT Mean'
    delta = random.uniform(factor * 0.1, factor * 0.2) 
    updated_factor = factor + delta
    dns_dataset[col] = dns_dataset['IAT Mean'] * updated_factor

---

## Adding the Label column:

In [27]:
# adding a label to the dataset
dns_dataset['Label'] = ATTACK_NAME

---

## Validate that the generated data looks valid by comparing the samples with the generated dataset:

In [28]:
dns_samples

,A Record Count,AAAA Record Count,CName Record Count,TXT Record Count,MX Record Count,DF Flag Count,Average Response Data Length,Min Response Data Length,Max Response Data Length,Average Domain Name Length,Min Domain Name Length,Max Domain Name Length,Average Sub Domain Name Length,Min Sub Domain Name Length,Max Sub Domain Name Length,Average Packet Length,Min Packet Length,Max Packet Length,Number of Domian Names,Number of Sub Domian Names,Total Length of Fwd Packet,Total Length of Bwd Packet,Total Number of Packets,Flow Duration,IAT Max,IAT Mean,IAT Std
0,0,0,150,84,116,174,39.758621,34,70,181.363636,42,241,32.309848,14.333333,39.833333,268.742857,101,386,88,276,44448,34912,350,17.115152,1.102749,0.049041,0.153262
1,0,0,94,68,60,110,39.550000,34,70,149.000000,42,241,28.125000,14.333333,39.833333,233.810811,101,357,56,148,23990,18592,222,14.696482,0.770567,0.066500,0.158759
2,0,0,94,96,136,162,38.127660,34,52,185.609756,42,241,32.740854,14.333333,39.833333,272.122699,101,357,82,262,41792,33228,326,14.316078,0.990149,0.044049,0.145406
3,0,0,100,128,88,158,38.175439,34,70,170.493671,42,241,30.845992,14.333333,39.833333,256.006329,101,357,79,234,38002,29624,316,16.216136,0.999400,0.051480,0.151140
4,0,0,136,122,88,172,38.406250,34,44,207.091954,42,241,35.493295,14.333333,39.833333,292.161850,101,357,87,306,47564,38992,346,11.440871,0.990594,0.033162,0.099968
5,0,0,56,90,68,106,37.388889,34,44,155.870370,42,241,28.910494,14.333333,39.833333,240.252336,101,357,54,149,23756,18670,214,13.661421,0.801823,0.064138,0.165087
6,0,0,74,60,80,106,38.909091,34,52,108.611111,42,241,22.862654,14.333333,39.833333,194.897196,101,357,54,111,19154,13566,214,19.716813,0.987562,0.092567,0.255850
7,0,0,66,52,72,94,39.655172,34,60,132.375000,42,241,25.881944,14.333333,39.833333,219.200000,101,357,48,116,19328,14340,190,13.953869,0.996097,0.073830,0.189524
8,0,0,72,52,80,102,38.645161,34,42,84.529412,42,241,19.767974,14.333333,39.833333,170.578431,101,357,51,86,15874,10356,204,21.172616,0.993989,0.104299,0.275736
9,0,0,60,68,96,112,38.875000,34,60,119.464286,42,241,24.232143,14.333333,39.833333,205.500000,101,357,56,124,21340,15284,224,18.313704,0.979127,0.082124,0.187037


In [29]:
dns_dataset

,A Record Count,AAAA Record Count,CName Record Count,TXT Record Count,MX Record Count,DF Flag Count,Average Response Data Length,Min Response Data Length,Max Response Data Length,Average Domain Name Length,Min Domain Name Length,Max Domain Name Length,Average Sub Domain Name Length,Min Sub Domain Name Length,Max Sub Domain Name Length,Average Packet Length,Min Packet Length,Max Packet Length,Number of Domian Names,Number of Sub Domian Names,Total Length of Fwd Packet,Total Length of Bwd Packet,Total Number of Packets,Flow Duration,IAT Max,IAT Mean,IAT Std,Label
0,0,0,105,121,116,167,40,33,57,124.640990,47,237,20,12,35,195.141639,115,398,52.087743,161.903345,24968.744358,14004.472088,152.745326,12.289626,0.663632,0.044249,0.140526,DNS
1,0,0,53,59,150,142,38,37,55,169.301178,41,245,28,12,44,270.967873,101,380,59.613649,175.924356,28044.682234,28250.244604,294.898059,23.234139,1.254630,0.083655,0.265670,DNS
2,0,0,115,146,67,175,37,30,75,121.304754,43,257,19,15,41,137.533201,105,348,52.578456,107.582658,18235.397200,13175.234831,147.316322,30.474223,1.645590,0.109724,0.348457,DNS
3,0,0,84,79,93,152,40,36,51,95.355032,45,271,21,12,43,190.074515,91,336,54.129374,154.398343,26968.657308,20971.090058,229.765860,27.183888,1.467914,0.097877,0.310834,DNS
4,0,0,100,167,55,134,37,37,57,164.069280,37,227,36,15,43,256.237147,99,356,71.148402,203.192695,36389.583265,25285.933347,384.257905,11.356354,0.613236,0.040889,0.129854,DNS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,0,0,157,69,108,113,43,36,78,95.152610,42,260,21,12,40,195.085999,111,362,41.722544,127.218142,21070.434026,15073.859726,175.568829,8.964194,0.484061,0.032276,0.102501,DNS
39996,0,0,141,168,111,144,37,34,42,140.575055,37,255,24,12,41,169.650092,106,391,49.209418,192.871261,24335.411247,24512.009466,259.021497,29.001978,1.566089,0.104423,0.331623,DNS
39997,0,0,135,53,124,121,39,30,63,115.138049,47,270,26,12,41,193.516657,114,331,53.211755,158.212101,32922.548683,24974.680187,277.167783,30.711946,1.658427,0.110579,0.351175,DNS
39998,0,0,102,128,144,171,44,36,70,140.695768,42,274,29,13,42,202.621234,91,360,79.674437,217.572656,27312.971880,20605.718660,302.169624,29.544822,1.595403,0.106377,0.337830,DNS


In [33]:
dns_samples.describe()

,A Record Count,AAAA Record Count,CName Record Count,TXT Record Count,MX Record Count,DF Flag Count,Average Response Data Length,Min Response Data Length,Max Response Data Length,Average Domain Name Length,Min Domain Name Length,Max Domain Name Length,Average Sub Domain Name Length,Min Sub Domain Name Length,Max Sub Domain Name Length,Average Packet Length,Min Packet Length,Max Packet Length,Number of Domian Names,Number of Sub Domian Names,Total Length of Fwd Packet,Total Length of Bwd Packet,Total Number of Packets,Flow Duration,IAT Max,IAT Mean,IAT Std
count,20.0,20.0,20.000000,20.000000,20.000000,20.000000,20.000000,20.0,20.000000,20.000000,20.0,20.0,20.000000,2.000000e+01,2.000000e+01,20.000000,20.0,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000
mean,0.0,0.0,88.600000,87.300000,90.200000,132.500000,38.511953,34.0,54.200000,151.014408,42.0,241.0,28.317440,1.433333e+01,3.983333e+01,236.795765,101.0,358.450000,66.800000,186.200000,30354.600000,23409.800000,266.100000,16.006378,0.931769,0.064669,0.175722
std,0.0,0.0,31.147189,28.181928,21.607991,31.768074,0.778566,0.0,9.058872,37.134279,0.0,0.0,4.773938,1.822504e-15,7.290015e-15,37.170469,0.0,6.484597,15.813385,76.963833,11632.193025,9970.843324,63.386783,2.757227,0.109439,0.022172,0.053685
min,0.0,0.0,42.000000,52.000000,54.000000,94.000000,37.311475,34.0,42.000000,84.529412,42.0,241.0,19.767974,1.433333e+01,3.983333e+01,170.578431,101.0,357.000000,48.000000,84.000000,15286.000000,10120.000000,190.000000,11.440871,0.764955,0.033162,0.099968
25%,0.0,0.0,63.000000,68.000000,78.000000,105.500000,37.981744,34.0,50.000000,119.124530,42.0,241.0,24.188727,1.433333e+01,3.983333e+01,205.268805,101.0,357.000000,53.750000,122.000000,20837.000000,15048.000000,213.000000,13.880757,0.794057,0.046086,0.140232
50%,0.0,0.0,89.000000,82.000000,88.000000,112.000000,38.525706,34.0,52.000000,153.586129,42.0,241.0,28.667511,1.433333e+01,3.983333e+01,239.054740,101.0,357.000000,56.500000,148.500000,23888.000000,18631.000000,225.000000,15.517295,0.989264,0.063071,0.161923
75%,0.0,0.0,103.000000,102.000000,108.000000,165.000000,39.074074,34.0,60.000000,181.570346,42.0,241.0,32.304397,1.433333e+01,3.983333e+01,267.810714,101.0,357.000000,82.500000,262.500000,42089.000000,33287.000000,330.000000,18.407313,0.994516,0.082358,0.200219
max,0.0,0.0,150.000000,148.000000,136.000000,174.000000,39.758621,34.0,70.000000,207.091954,42.0,241.0,35.493295,1.433333e+01,3.983333e+01,292.161850,101.0,386.000000,88.000000,306.000000,47564.000000,38992.000000,350.000000,21.172616,1.102749,0.104299,0.275736


In [30]:
dns_dataset.describe()

,A Record Count,AAAA Record Count,CName Record Count,TXT Record Count,MX Record Count,DF Flag Count,Average Response Data Length,Min Response Data Length,Max Response Data Length,Average Domain Name Length,Min Domain Name Length,Max Domain Name Length,Average Sub Domain Name Length,Min Sub Domain Name Length,Max Sub Domain Name Length,Average Packet Length,Min Packet Length,Max Packet Length,Number of Domian Names,Number of Sub Domian Names,Total Length of Fwd Packet,Total Length of Bwd Packet,Total Number of Packets,Flow Duration,IAT Max,IAT Mean,IAT Std
count,40000.0,40000.0,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000
mean,0.0,0.0,104.180625,107.361475,101.648775,141.818150,38.509400,34.001325,57.893425,151.201827,41.979425,245.986150,28.019650,13.503700,39.470725,233.840375,102.466100,381.354925,66.504887,190.373226,30926.225696,24017.797972,264.233406,20.803002,1.123350,0.074902,0.237872
std,0.0,0.0,38.854749,35.699506,31.228151,33.522538,3.443451,2.569492,12.381617,42.152744,3.162910,17.622703,6.639628,1.116655,2.882456,64.944786,7.524687,35.150965,18.476885,52.839907,8632.683148,6719.318770,73.667922,7.233231,0.390590,0.026044,0.082708
min,0.0,0.0,37.000000,46.000000,48.000000,84.000000,33.000000,30.000000,37.000000,73.404241,37.000000,216.000000,17.000000,12.000000,35.000000,113.553787,90.000000,321.000000,32.268503,92.523798,15013.256540,11656.534473,128.542992,8.290335,0.447673,0.029850,0.094796
25%,0.0,0.0,71.000000,76.000000,75.000000,113.000000,36.000000,32.000000,47.000000,117.591558,39.000000,231.000000,22.000000,13.000000,37.000000,182.142247,96.000000,351.000000,51.845746,148.352842,24084.109039,18659.562941,205.375852,14.527377,0.784470,0.052306,0.166113
50%,0.0,0.0,104.000000,107.000000,102.000000,142.000000,39.000000,34.000000,58.000000,148.117289,42.000000,246.000000,28.000000,14.000000,39.000000,229.278186,102.000000,381.000000,65.002681,186.554912,30303.118743,23508.117998,259.065027,20.804462,1.123428,0.074907,0.237888
75%,0.0,0.0,138.000000,138.000000,129.000000,171.000000,42.000000,36.000000,69.000000,179.538967,45.000000,261.000000,34.000000,14.000000,42.000000,277.498605,109.000000,412.000000,78.955800,225.907475,36731.524056,28592.990748,314.070126,27.076707,1.462126,0.097491,0.309608
max,0.0,0.0,171.000000,169.000000,155.000000,199.000000,44.000000,38.000000,79.000000,252.542572,47.000000,276.000000,39.000000,15.000000,44.000000,390.637514,115.000000,442.000000,111.005065,318.335685,51642.024708,40098.554186,442.262824,33.311205,1.798785,0.119938,0.380896


---

## At the end we save the dataset as a CSV file

In [31]:
print(f'Attack Dataset Shape: {dns_dataset.shape}')

Attack Dataset Shape: (40000, 28)


In [ ]:
# save the dataset
dns_dataset.to_csv('dns_active_dataset.csv', index=False)